# Forecast of Australian beer production
For the purpouse of AI Tech Trainee time-series session

## Forecast of Australian beer production 

**The Story**
- Lets go back in time and space. It is as last date of the year 1969 and you live in Australia.
- You're a passionate beer drinker and you made a bet with your friends that you're going to drink one milionth of Australian overall beer production each month for the next 24 months. You can choose one friend to help you with the consumption. 
- If you succeed, all the bills will be covered for you. If you fail on any month, you'll need be be sober for the next 24 months.
- Now, you woke up on New Year's Eve with a headache and imediatelly remembered your yesterday bet.
- Luckily, you have an access to the relevant data (from 1956-1969), PC, and a strong thirst.
- Do you have any chance of winning the bet? How much beer do you need to drink each month?

**TL;DR story:**
- you have monthly data on Australian beer production starting 1956 till 1969 (180 months)
- you want to create a forecast for years 1970-1971 (24 months)


**Plan:**
- inspect the data
- explore patterns in the historical data
- make predictions using:
    - Naive models
    - Exponentials Smoothing models
    - tuned Exponentials Smoothing models


![img1](https://images.fineartamerica.com/images/artworkimages/mediumlarge/1/australian-beer-tap-gej-jones.jpg)


In [ ]:
# Source libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import warnings

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = 15, 7

# Importing Data
- Dataset: Australian beer production on 1956 - 1995

In [ ]:
df_raw = pd.read_csv("/kaggle/input/time-series-datasets/monthly-beer-production-in-austr.csv", 
                   parse_dates=['Month'], index_col='Month')
df_raw.columns = ['Production']

# Select only data till 1970 (including)
df = df_raw.loc[df_raw.index < '1970']

# Data Inspection


In [ ]:
df

In [ ]:
df.describe()

In [ ]:
# full time series
df.plot(figsize=(15, 7))
plt.title('Beer Production')
plt.ylabel('megalitres')
plt.show()

Data looks fine, no missing values, no outliers -> we can proceed with modeling.

## Time Series Decomposition

Here we're going to use a classical decomposition approach: 
1. compute trend using moving average  (-> loose first and last 6 months)
2. detrend data  (subtract trend line from the data)
3. compute average seasonal effect of each month

or use `seasonal_decompose` function that does it for you.

More on the topic here: https://otexts.com/fpp2/classical-decomposition.html

In [ ]:
# Compute centred moving averages
df_ma = df.rolling(window=12, center=True).mean()

# Plot including moving average (orange line)
plt.figure(figsize=(15, 7))
plt.plot(df)
plt.plot(df_ma)
plt.title('Beer Production')
plt.ylabel('megalitres')
plt.show()

In [ ]:
# Detrended data
detrended_df = df - df_ma

plt.figure(figsize=(15, 7))
plt.plot(detrended_df)
plt.title('Beer Production')
plt.ylabel('megalitres')
plt.show()

In [ ]:
# Yearly seasonality plot
detrended_df['year'] = detrended_df.index.year
detrended_df['month'] = detrended_df.index.month
df_pivot = pd.pivot_table(detrended_df, values='Production', index='month', columns='year', aggfunc='mean')
df_pivot.plot(figsize=(15, 7))
plt.legend().remove()
plt.xlabel('Month')
plt.ylabel('Beer Production')
plt.show()

In [ ]:
# Average seasonal effect computed
detrended_df.groupby('month')['Production'].mean()

In [ ]:
# Or we can do the same thing bit more easily with these few lines
decomposition = seasonal_decompose(df['Production'], freq=12, model='additive')

decomposition.plot()
plt.show();

# Forecasting models
- So far we only looked in to the past, trying to see its patterns
- Now, we'll need to focus on using the past information to predict the future months

In [ ]:
# Basic prediction settings
seasonal_period = 12     # what is the length of seasonal cycle
prediction_horizon = 24  # how many months to predict

prediction_index = pd.date_range("1970-01-01", periods=prediction_horizon, freq="MS")

###  Naïve average model   
- **average all values**

In [ ]:
predictionN1 = df.mean()
predictionN1 = predictionN1.repeat(prediction_horizon).to_frame()
predictionN1.index = prediction_index

plt.plot(df, label='training')
plt.plot(predictionN1, label='prediction')

###  Simple Naïve model – random walk  ()
- **use latest observation**

In [ ]:
predictionN2 = df.iloc[-1]
predictionN2 = predictionN2.repeat(prediction_horizon).to_frame()
predictionN2.index = prediction_index

plt.plot(df, label='training')
plt.plot(predictionN2, label='prediction')

###  Naïve seasonal model
- **repeat values 12 months back**

In [ ]:
predictionN3 = df.iloc[-seasonal_period:]
predictionN3 = predictionN3.append(predictionN3)
predictionN3.index = prediction_index

plt.plot(df, label='training')
plt.plot(predictionN3, label='prediction')

## Exponential smoothing models

- Starting from simple versions of the model to more complex ones:
  - Simple Exponential Smoothing
  - Double Exponential Smoothing
  - Tripple Exponential Smoothing  (a.k.a. Holt-Winters model)

No parameter tuning at this stage.

In [ ]:
# Simple Exponential Smoothing  (level only)
exp_model_1 = (ExponentialSmoothing(df,
                                   trend=None, 
                                   seasonal=None,
                                   seasonal_periods=None)
               .fit(smoothing_level=0.3))

predictionE1 = exp_model_1.forecast(prediction_horizon)

plt.plot(df)
plt.plot(predictionE1)

In [ ]:
# Double Exponential Smoothing  (level + trend)
exp_model_2 = (ExponentialSmoothing(df,
                                   trend='add', 
                                   seasonal=None,
                                   seasonal_periods=None)
               .fit(smoothing_level=0.3,
                    smoothing_trend=0.3))

predictionE2 = exp_model_2.forecast(prediction_horizon)

plt.plot(df)
plt.plot(predictionE2)

In [ ]:
# Triple Exponential Smoothing  (level + trend + seasonality)
exp_model_3 = (ExponentialSmoothing(df,
                                   trend='add', 
                                   seasonal='add',
                                   seasonal_periods=seasonal_period).
               fit(smoothing_level=0.3,
                   smoothing_trend=0.3,
                   smoothing_seasonal=0.3))

predictionE3 = exp_model_3.forecast(prediction_horizon)

plt.plot(df)
plt.plot(predictionE3)

## Parameter tuning and error measures (accuracy)
- Instead of manually tweaking the parameters of Exponential smoothing, you better automate the process of choosing the best parameter
- First we need to select some data for the tuning (e.g. last 12 or 24 months)

In [ ]:
tuning_test_period = 12

# Test-train data split
df_train = df[df.index < '1969']
df_test = df[df.index >= '1969']

plt.plot(df_train)
plt.plot(df_test, 'g-')

In [ ]:
# Simple Exponential Smoothing  (level only) with Parameter tuning

smoothing_levels = [0.1, 0.3, 0.5, 0.7, 0.9]

plt.plot(df_train)
plt.plot(df_test, 'g-')

for i in smoothing_levels:
    exp_model_t1 = (ExponentialSmoothing(df_train,
                                       trend=None, 
                                       seasonal=None,
                                       seasonal_periods=None)
                   .fit(smoothing_level=i))

    predictionEt1 = exp_model_t1.forecast(tuning_test_period)

    plt.plot(predictionEt1)
    
    print(f'smoothing_levels={i}  -  {np.sqrt(mean_squared_error(df_test, predictionEt1))}')

In [ ]:
# Simple Exponential Smoothing  (level only)  -  USE tuned parameters
exp_model_1 = (ExponentialSmoothing(df,
                                   trend=None, 
                                   seasonal=None,
                                   seasonal_periods=None)
               .fit(smoothing_level=0.1))

predictionET1 = exp_model_1.forecast(prediction_horizon)

plt.plot(df)
plt.plot(predictionET1)

In [ ]:
# Double Exponential Smoothing  (level only) with Parameter tuning

smoothing_levels = [0.1, 0.3, 0.5, 0.7, 0.9]
smoothing_trends = [0.1, 0.3, 0.5, 0.7, 0.9]

plt.plot(df_train)
plt.plot(df_test, 'g-')

for i in smoothing_levels:
    for j in smoothing_trends:
        exp_model_t2 = (ExponentialSmoothing(df_train,
                                           trend='add', 
                                           seasonal=None,
                                           seasonal_periods=None)
                       .fit(smoothing_level=i,
                           smoothing_trend=j))

        predictionEt2 = exp_model_t2.forecast(tuning_test_period)

        plt.plot(predictionEt2)

        print(f'level={i} & trend={j}  -  {np.sqrt(mean_squared_error(df_test, predictionEt2))}')

In [ ]:
# Double Exponential Smoothing  (level + trend)  -  USE tuned parameters
exp_model_T2 = (ExponentialSmoothing(df,
                                   trend='add', 
                                   seasonal=None,
                                   seasonal_periods=None)
               .fit(smoothing_level=0.1,
                    smoothing_trend=0.5))

predictionET2 = exp_model_T2.forecast(prediction_horizon)

plt.plot(df)
plt.plot(predictionET2)

In [ ]:
# Triple Exponential Smoothing  (level only) with Parameter tuning

smoothing_levels = [0.1, 0.3, 0.5, 0.7, 0.9]
smoothing_trends = [0.1, 0.3, 0.5, 0.7, 0.9]
smoothing_seasonal = [0.1, 0.3, 0.5, 0.7, 0.9]

plt.plot(df_train)
plt.plot(df_test, 'g-')

for i in smoothing_levels:
    for j in smoothing_trends:
        for k in smoothing_seasonal:
            exp_model_t3 = (ExponentialSmoothing(df_train,
                                               trend='add', 
                                               seasonal='add',
                                               seasonal_periods=seasonal_period)
                           .fit(smoothing_level=i,
                               smoothing_trend=j,
                               smoothing_seasonal=k))

            predictionEt3 = exp_model_t3.forecast(tuning_test_period)

            plt.plot(predictionEt3)

            print(f'level={i} & trend={j} & seasonal={k} -  {np.sqrt(mean_squared_error(df_test, predictionEt3))}')

In [ ]:
# Triple Exponential Smoothing  (level + trend + seasonality)  -  USE tuned parameters
exp_model_T3 = (ExponentialSmoothing(df,
                                   trend='add', 
                                   seasonal='add',
                                   seasonal_periods=seasonal_period).
               fit(smoothing_level=0.7,
                   smoothing_trend=0.5,
                   smoothing_seasonal=0.5))

predictionET3 = exp_model_T3.forecast(prediction_horizon)

plt.plot(df)
plt.plot(predictionET3)

# Final predictions
- on the tuning test data we could see that the Tripple exponential smoothing were doing the best from the Exponential smoothing model family. Therefore, we can assume that their predictions will be the most accurate also for future months (1970 and further)

In [ ]:
predictionET3

# Conclusion

- now we have a good idea about the future of Australian beer production
- do you think you would stand a chance to the bet? 

Fast forward to the year 1972, this is how the beer production developed.

### Now we can check if our predictions back in 1969 were any good

In [ ]:
# full time series
df_raw.loc[df_raw.index < '1972'].plot(figsize=(15, 7))
predictionET3.plot()
plt.title('Beer Production')
plt.ylabel('megalitres')
plt.show()

In [ ]:
# Final prediction evaluation
df_test = df_raw[(df_raw.index >= '1970-01-01') & (df_raw.index <= '1971-12-31')]

print("NAIVE MODELS")
for pred in [predictionN1, predictionN2, predictionN3]:
    print(f'RMSE={np.sqrt(mean_squared_error(df_test, pred))}, MAPE={mean_absolute_percentage_error(df_test, pred)}')   

print("\nEXPONENTIAL SMOOTHING MODELS")
for pred in [predictionE1, predictionE2, predictionE3]:
    print(f'RMSE={np.sqrt(mean_squared_error(df_test, pred))}, MAPE={mean_absolute_percentage_error(df_test, pred)}')   

print("\nTUNED EXPONENTIAL SMOOTHING MODELS")
for pred in [predictionET1, predictionET2, predictionET3]:
    print(f'RMSE={np.sqrt(mean_squared_error(df_test, pred))}, MAPE={mean_absolute_percentage_error(df_test, pred)}')   


In [ ]:
# full time series
df_raw.plot(figsize=(15, 7))
predictionET3.plot()
plt.title('Beer Production')
plt.ylabel('megalitres')
plt.show()



![img2](https://i.dailymail.co.uk/1s/2020/02/18/23/24907772-0-image-a-16_1582069652685.jpg)

## What else could be done to improve predictions
- different parameter choices (of Exponential models), also trying multiplicative trend and/or seasonality
- different test data length, or better do a cross validation (e.g. https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection)
- try different models (e.g. SARIMA, FB Prophet)
- and many more...